In [ ]:
# run to pretty-print results
import numpy as np

np.set_printoptions(precision=5, suppress=True)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

The amd package contains functions for caluclating and comparing AMDs/PDDs, as well as .cif reading functionality provided by either ase or ccdc (license required).

If not installed already, get amd via pip:
```shell
$ pip install average-minimum-distance
```

To read .cifs it is recommended you also pip install ase, the default back end .cif reader. If you have ccdc installed and a valid license, it can be used instead. ccdc also enables searching the CSD by refcode. 

**Before running any cells below, import amd by running**

In [ ]:
import amd

# Reading cifs

## With amd.CifReader

To read cifs with amd, create an ```amd.CifReader``` object and pass the path to the cif as below. (The .cifs in this notebook can be found in the tests folder of this project). The CifReader will try to extract the correct information; if a structure cannot be read by default it is skipped by the reader and a warning is printed. This returns an iterator which can be looped over to get all structures from a file. It yields ```PeriodicSet``` objects, which can be handed directly to the AMD/PDD calculator functions below. ```PeriodicSet```s basically have only 3 attributes, a ```name```, ```motif``` and ```cell```.

In [ ]:
# one cif, many structures

reader = amd.CifReader('T2_experimental.cif')
for periodic_set in reader:
    print(periodic_set.name, periodic_set.motif.shape[0])   # print name & number of motif points
    periodic_set.cell                                       # access unit cell

# if you don't care about lazy reading and just want a list of structures,
exp_structures = list(amd.CifReader('T2_experimental.cif'))

Below, setting ```remove_hydrogens``` to True will ignore all Hydrogen atoms in the file. The CifReader has a few optional arguments, but this is the only stable one and others (like ```heaviest_component``` below) should be only be changed with caution.

In [ ]:
# one cif, one structure

gamma = list(amd.CifReader('T2_gamma.cif', remove_hydrogens=True))[0]

T2_alpha.cif has a solvent molecule with disorder. Setting ```heaviest_component``` to True (ccdc only) will attempt to remove the solvent by only extracting the heaviest molecule in the asymmetric unit. Use with caution; if for example the asymmetric unit is already expanded in your .cif or you have a co-crystal this results in unintended behaviour.

In [ ]:
# an odd case where we need ccdc. Will error if not installed or licensed

reader = amd.CifReader('T2_alpha.cif', reader='ccdc', heaviest_component=True)
alpha = list(reader)[0]
print(alpha)

The core object representing a crystal structure in ase is called ```Atoms```. Using ase, you can read ```Atoms``` from a cif or extract the raw data:

# Calculating AMDs and PDDs

The main calculator functions ```amd.amd``` and ```amd.pdd``` accept two arguments, a periodic set and an integer ```k```. The periodic set can be either a ```PeriodicSet``` object as given by a CifReader, or a pair of numpy arrays (motif, cell).

In [ ]:
k = 100

# one AMD from a .cif with one structure
gamma = list(amd.CifReader('T2_gamma.cif'))[0]
gamma_amd = amd.amd(gamma, k) 
print(gamma_amd)

# list of amds from a .cif with many structures
exp_structures = amd.CifReader('T2_experimental.cif')
experimental_amds = [amd.amd(periodic_set, k) for periodic_set in exp_structures] # list of AMDs

In [ ]:
k = 100

# the pdd interface is the same as amd
gamma = list(amd.CifReader('T2_gamma.cif'))[0]
gamma = next(iter(amd.CifReader('T2_gamma.cif')))
# print(gamma)
print(gamma)
print('unit cell:\n', gamma.cell)
print('5 points of the motif:\n', gamma.motif[:5])
gamma_pdd = amd.pdd(gamma, k)

print('PDD:\n', gamma_pdd)

In [ ]:
import numpy as np

k = 100

# from a tuple (motif, cell) of numpy arrays
motif = np.array([[0,0,0]]) # one point at the origin
cell = np.identity(3)       # unit cell = identity (cube with unit edges)
cubic_pdd = amd.pdd((motif, cell), k)

print(cubic_pdd)

# Comparing AMDs and PDDs

There are several functions which compare by AMD or PDD. The names 'cdist' and 'pdist' come from scipy: cdist is for comparing one set against another, whereas pdist takes one set and compares it pairwise.  So to compare with AMDs, use ```amd.amd_pdist``` or ```amd.amd_cdist``` and to compare with PDDs use ```amd.pdd_pdist``` or ```amd.pdd_cdist```. Example:

In [ ]:
# comapres T2-gamma to all experimental structures by AMD, k=100.

import numpy as np

exp_structures = list(amd.CifReader('T2_experimental.cif'))
gamma = list(amd.CifReader('T2_gamma.cif'))[0]

k = 100
exp_amds = [amd.amd(s, k) for s in exp_structures]
gamma_amd = amd.amd(gamma, k)

dm = amd.amd_cdist(gamma_amd, exp_amds)  # compare gamma to exp structures
closest_indices = np.argsort(dm)[0]  # get indices of smallest elements i.e closest
for i in closest_indices:
    print(exp_structures[i].name, dm[0][i])  # print structs in order of ditance


The output of these functions mimic scipy's [cdist](scipy.spatial.distance.cdist) and [pdist](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html), cdist returns a 2D distance matrix and pdist returns a 'condensed distance vector' ([see scipy's squareform](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.squareform.html)).

These functions take the same optional arguments. If you want to change the defaults to see the effect on comparisons, details are in the readme and documentation. Briefly, ```metric``` (default 'chebyshev', aka l-infinity) is the metric used to compare AMDs or the rows of PDDs, it can be any metric accepted by [scipy's cdist function](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html). ```k``` (default None) is the value of k to use when comparing, it just truncates the inputs to a smaller length if an int, but can also be a ```range``` which compares over a range of k and collapses the resulting sequences of distances with a norm optionally specified with ```ord```, which accepts any ```ord``` accepted by [numpy's norm function](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html). 

 The functions that use PDD also take a parameter ```verbose``` which if true will print an ETA to the terminal.

In [ ]:
k = 100

exp_structures = list(amd.CifReader('T2_experimental.cif'))
experimental_pdds = [amd.pdd(periodic_set, k) for periodic_set in exp_structures]

# compare experimental structures pairwise
# returns a condensed distance matrix
cdm = amd.pdd_pdist(experimental_amds)